## Importing the neccessary libraries 

## Now we are going to record the audio and convert this audio file into a text file

In [10]:
import pyaudio
import wave
import speech_recognition as sr
import pyttsx3
# Initializing The Recognizer
r = sr.Recognizer()
# Parameters
FORMAT = pyaudio.paInt16  # Audio format
CHANNELS = 1  # Number of channels
RATE = 44100  # Sample rate (Hz)
CHUNK = 1024  # Chunk size (number of frames per buffer)
RECORD_SECONDS = 10  # Duration of the recording (seconds)
OUTPUT_FILENAME = "recorded_audio.wav"  # Output filename

try:
    # Initialize PyAudio
    audio = pyaudio.PyAudio()

    # Open stream
    stream = audio.open(format=FORMAT, channels=CHANNELS,
                        rate=RATE, input=True,
                        frames_per_buffer=CHUNK)

    print("Recording...")

    frames = []

    # Record data
    for i in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
        data = stream.read(CHUNK)
        frames.append(data)

    print("Recording finished.")

    # Stop and close the stream
    stream.stop_stream()
    stream.close()
    audio.terminate()

    # Save the recorded data as a WAV file
    with wave.open(OUTPUT_FILENAME, 'wb') as wf:
        wf.setnchannels(CHANNELS)
        wf.setsampwidth(audio.get_sample_size(FORMAT))
        wf.setframerate(RATE)
        wf.writeframes(b''.join(frames))

    print(f"Audio recorded and saved as {OUTPUT_FILENAME}")
    with sr.AudioFile(OUTPUT_FILENAME) as source:
        audio = r.record(source)  # Read the entire audio file

        try:
            # Recognize the speech using Google Web Speech API
            text = r.recognize_google(audio)
            print("Transcription: " + text)
            # Append the transcription to a text file
            with open("transcription.txt", "a") as f:
                f.write(text + "\n")
            # Save the transcription to a text file
            with open("transcription.txt", "w") as f:
                f.write(text)
                
        except sr.UnknownValueError:
            print("Google Speech Recognition could not understand the audio")
        except sr.RequestError as e:
            print("Could not request results from Google Speech Recognition service; {0}".format(e))
except OSError as e:
    print(f"OSError encountered: {e}")

OSError encountered: [Errno -9996] Invalid input device (no default output device)


updated for continuos recording

In [7]:
import pyaudio
import wave
import speech_recognition as sr
import threading
import signal
import sys

# Initializing The Recognizer
r = sr.Recognizer()

# Parameters
FORMAT = pyaudio.paInt16  # Audio format
CHANNELS = 1  # Number of channels
RATE = 44100  # Sample rate (Hz)
CHUNK = 1024  # Chunk size (number of frames per buffer)
OUTPUT_FILENAME = "recorded_audio.wav"  # Output filename

stop_event = threading.Event()

def record_and_transcribe():
    try:
        # Initialize PyAudio
        audio = pyaudio.PyAudio()

        # Open stream
        stream = audio.open(format=FORMAT, channels=CHANNELS,
                            rate=RATE, input=True,
                            frames_per_buffer=CHUNK)

        print("Recording...")

        frames = []

        while not stop_event.is_set():
            data = stream.read(CHUNK)
            frames.append(data)
            audio_data = b''.join(frames)

            # Save the recorded data as a WAV file
            with wave.open(OUTPUT_FILENAME, 'wb') as wf:
                wf.setnchannels(CHANNELS)
                wf.setsampwidth(audio.get_sample_size(FORMAT))
                wf.setframerate(RATE)
                wf.writeframes(audio_data)

            with sr.AudioFile(OUTPUT_FILENAME) as source:
                audio = r.record(source)

                try:
                    # Recognize the speech using Google Web Speech API
                    text = r.recognize_google(audio)
                    print("Transcription: " + text)
                    # Append the transcription to a text file
                    with open("transcription.txt", "a") as f:
                        f.write(text + "\n")

                except sr.UnknownValueError:
                    print("Google Speech Recognition could not understand the audio")
                except sr.RequestError as e:
                    print("Could not request results from Google Speech Recognition service; {0}".format(e))

        # Stop and close the stream
        stream.stop_stream()
        stream.close()
        audio.terminate()

    except OSError as e:
        print(f"OSError encountered: {e}")

def signal_handler(sig, frame):
    print("Interrupt received, stopping...")
    stop_event.set()
    sys.exit(0)

if __name__ == "__main__":
    signal.signal(signal.SIGINT, signal_handler)
    record_and_transcribe()


OSError encountered: [Errno -9996] Invalid input device (no default output device)
